In [2]:
# 代码1.1 Tensorflow中基于张量的矩阵乘法

In [3]:
import tensorflow as tf

In [4]:
a = tf.constant([[1,2],[1,2]])

2022-02-21 21:41:08.449761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-21 21:41:08.530404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: /usr/local/cuda-11.5/lib64/libcudnn.so.8: file too short; LD_LIBRARY_PATH: :/usr/local/cuda-11.5/lib64
2022-02-21 21:41:08.530428: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-02-21 21:41:08.531425: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI 

In [5]:
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [1, 2]], dtype=int32)>

In [6]:
b = tf.constant([[1],[2]])

In [7]:
b

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[1],
       [2]], dtype=int32)>

In [8]:
c = tf.matmul(a,b)

In [9]:
c

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[5],
       [5]], dtype=int32)>

In [9]:
# 代码1.2 Tensorflow和Tensorlayer中的梯度计算

In [10]:
import tensorflow as tf
import tensorlayer as tl

In [11]:
def train(model, dataset, optimizer):
    for x, y in dataset:
        with tf.GradientTape as tape:
            prediction = model(x)
            loss = loss_fc(prediction, y)
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))

In [12]:
# 代码1.3 静态模型样例：多层感知器(MLP)

In [13]:
import tensorflow as tf
from tensorlayer.layers import Input, Dense
from tensorlayer.models import Model

In [14]:
def get_mlp_model(inputs_shape):
    ni = Input(inputs_shape)
    nn = Dense(n_units=800, act=tf.nn.relu)(ni)
    nn = Dense(n_units=800, act=tf.nn.relu)(nn)
    nn = Dense(n_units=10, act=tf.nn.relu)(nn)
    
    M = Model(inputs=ni, outputs=nn)
    return M

In [15]:
MLP = get_mlp_model([None, 784])
MLP.eval()
outputs = MLP(data)

[TL] Input  _inputlayer_1: [None, 784]
[TL] Dense  dense_1: 800 relu
[TL] Dense  dense_2: 800 relu
[TL] Dense  dense_3: 10 relu


NameError: name 'data' is not defined

In [ ]:
# 代码1.4 动态模型样例：多层感知器(MLP)

In [16]:
import tensorflow as tf
from tensorlayer.layers import Input, Dense
from tensorlayer.models import Model

In [17]:
class MLPModel(Model):
    def __init__(self):
        super(MLPModel, self).__init__()
        
        self.dense1 = Dense(n_units=800, act=tf.nn.relu, in_channels=784)
        self.dense2 = Dense(n_units=800, act=tf.nn.relu, in_channels=800)
        self.dense3 = Dense(n_units=10, act=tf.nn.relu, in_channels=800)
        
    def forward(self, x, foo=False):
        z = self.dense1(x)
        z = self.dense2(z)
        out = self.dense3(z)
        
        if foo:
            out = tf.nn.softmax(out)
            
        return out

In [18]:
MLP = MLPModel()
MLP.eval()

outputs_1 = MLP(data, foo=True)
outputs_2 = MLP(data, foo=False)

[TL] Dense  dense_4: 800 relu
[TL] Dense  dense_5: 800 relu
[TL] Dense  dense_6: 10 relu


NameError: name 'data' is not defined

In [19]:
# 自定义层

In [20]:
import tensorlayer as tl

x = tl.layers.Input([8, 3],name="inut")
y = tl.layers.Lambda(lambda x: 2*x)(x)
def customize_fn(input, foo):
    return foo*input
z = tl.layers.Lambda(customize_fn, fn_args={'foo': 42})(x) 

[TL] Input  inut: [8, 3]
[TL] Lambda  lambda_1: func: <function <lambda> at 0x7ff4cdab4290>, len_weights: 0
[TL] Lambda  lambda_2: func: <function customize_fn at 0x7ff4cdab4cb0>, len_weights: 0


In [21]:
import tensorflow as tf
import tensorlayer as tl

a = tf.Variable(1.0)
def customize_fn(x):
    return x + a
x = tl.layers.Input([8, 3], name="input")
y = tl.layers.Lambda(customize_fn, fn_weights=[a])(x)

[TL] Input  input: [8, 3]
[TL] Lambda  lambda_3: func: <function customize_fn at 0x7ff4cdad60e0>, len_weights: 1


In [22]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np

layers = [
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(5, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(1, activation=tf.identity)
]
perception = tf.keras.Sequential(layers)
_ = perception(np.random.random([10, 5]).astype(np.float32))

class CustomizeModel(tl.models.Model):
    def __init__(self):
        super(CustomizeModel, self).__init__()
        self.dense = tl.layers.Dense(in_channels=1, n_units=5)
        self.lambdalayer = tf.layers.Lambda(perception, perception.trainable_variables)
        
    def forward(self, x):
        z = self.dense(x)
        z = self.lambdalayer(z)
        return z

In [23]:
# 多层感知器：MNIST数据集上的图像分类

In [24]:
import tensorflow as tf
import tensorlayer as tl

X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=[-1, 784])

ni = tl.layers.Input([None, 784])
nn = tl.layers.Dropout(keep=0.8)(ni)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=800, act=tf.nn.relu)(nn)
nn = tl.layers.Dropout(keep=0.5)(nn)
nn = tl.layers.Dense(n_units=10, act=None)(nn)

network = tl.models.Model(inputs=ni, outputs=nn, name="mlp")

def acc(_logits, y_batch):
    return tf.reduce_mean(
        tf.cast(
            tf.equal(
                tf.argmax(_logits, 1),
                tf.convert_to_tensor(y_batch, tf.int64)),
        tf.float32),
        name='accuracy'
    )

tl.utils.fit(
    network, 
    train_op=tf.optimizers.Adam(learning_rate=0.0001),
    cost=tl.cost.cross_entropy,
    X_train=X_train, y_train=y_train,
    acc=acc,
    batch_size=256,
    n_epoch=20,
    X_val=X_val, y_val=y_val, eval_train=True,
)

tl.utils.test(
    network,
    acc=acc,
    X_test=X_test, y_test=y_test,
    batch_size=None,
    cost=tl.cost.cross_entropy,
)

network.save_weights("model.h5")

[TL] Load or Download MNIST > data/mnist
[TL] Downloading train-images-idx3-ubyte.gz...



100% (1211 of 1211) |####################| Elapsed Time: 0:03:41 ETA:  00:00:00

[TL] Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
[TL] data/mnist/train-images-idx3-ubyte.gz
[TL] Downloading train-labels-idx1-ubyte.gz...



100% (4 of 4) |##########################| Elapsed Time: 0:00:00 ETA:  00:00:00

[TL] Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
[TL] Downloading t10k-images-idx3-ubyte.gz...



100% (202 of 202) |######################| Elapsed Time: 0:00:22 ETA:  00:00:00

[TL] Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
[TL] data/mnist/t10k-images-idx3-ubyte.gz
[TL] Downloading t10k-labels-idx1-ubyte.gz...



100% (1 of 1) |##########################| Elapsed Time: 0:00:00 ETA:  00:00:00

[TL] Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
[TL] Input  _inputlayer_2: [None, 784]
[TL] Dropout dropout_1: keep: 0.800000 
[TL] Dense  dense_7: 800 relu
[TL] Dropout dropout_2: keep: 0.500000 
[TL] Dense  dense_8: 800 relu
[TL] Dropout dropout_3: keep: 0.500000 
[TL] Dense  dense_9: 10 No Activation
[TL] Finished! use `tensorboard --logdir=None/` to start tensorboard
[TL] Start training the network ...
[TL] Epoch 1 of 20 took 8.149116s
[TL]    train loss: 0.422891
[TL]    train acc: 0.878626
[TL] Epoch 5 of 20 took 3.685423s
[TL]    train loss: 0.194406
[TL]    train acc: 0.941526
[TL] Epoch 10 of 20 took 4.282970s
[TL]    train loss: 0.123684
[TL]    train acc: 0.962740
[TL] Epoch 15 of 20 took 4.393666s
[TL]    train loss: 0.089894
[TL]    train acc: 0.973357
[TL] Epoch 20 of 20 took 4.597843s
[TL]    train loss: 0.068480
[TL]    train acc: 0.979507
[TL] Total training time: 102.645401s
[TL] Start testing the network ...
[TL] [*] Saving TL weights into model.h5


In [25]:
# 卷积神经网络：CIFAR-10数据集上的图像分类

In [26]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import Input, Dense, Conv2d, BatchNorm2d, MaxPool2d, Flatten
from tensorlayer.models import Model
import numpy as np

def _fn_train(img, target):
    img = tl.prepro.crop(img, 24, 24, False)
    img = tl.prepro.flip_axis(img, is_random=True)
    img = tl.samplewise_norm(img)
    target = np.reshape(target, ())
    return img, target

train_ds = tl.data.CIFAR10(train_or_test='train', shape=(-1, 32, 32, 3))
train_dl = tl.data.Dataloader(train_ds, transforms=[_fn_train], shuffle=True, batch_size=batch_size, output_types=(np.float32, np.int32))

test_ds = tl.data.CIFAR10(train_or_test='test', shape=(-1, 32, 32, 3))
test_dl = tl.data.Dataloader(test_ds, batch_size=batch_size)

for X_batch, y_batch in train_dl:
    ...

AttributeError: module 'tensorlayer' has no attribute 'data'

In [27]:
def get_model_batchnorm(inputs_shape):
    W_init = tl.initializers.truncated_normal(stddev=5e-2)
    W_init2 = tl.initializers.truncated_normal(stddev=0.04)
    b_init2 = tl.initializers.constant(value=0.1)
    
    ni = Input(inputs_shape)
    
    nn = Conv2d(64, (5, 5), (1, 1), padding="SAME", W_init=W_init, b_init=None)(ni)
    nn = BatchNorm2d(decay=0.99, act=tf.nn.relu)(nn)
    nn = MaxPool2d((3, 3), (2, 2), padding="SAME")(nn)
    
    nn = Conv2d(64, (5, 5), (1, 1), padding="SAME", W_init=W_init, b_init=None)(ni)
    nn = BatchNorm2d(decay=0.99, act=tf.nn.relu)(nn)
    nn = MaxPool2d((3, 3), (2, 2), padding="SAME")(nn)
    
    nn = Flatten(nn)
    nn = Dense(n_units=384, act=tf.nn.relu, W_init=W_init2, b_init=b_init2)(nn)
    nn = Dense(n_units=193, act=tf.nn.relu, W_init=W_init2, b_init=b_init2)(nn)
    nn = Dense(n_units=10, act=None, W_init=W_init2)(nn)
    
    M = Model(inputs=ni, outputs=nn, name="cnn")
    return M

In [28]:
# 序列到序列模型：聊天机器人

In [29]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.models import Seq2seq

model = Seq2seq(
    decoder_seq_length=decoder_seq_length,
    cell_enc=tf.keras.layers.GRUCell,
    cell_dec=tf.keras.layers.GRUCell,
    n_layer=3,
    n_units=256,
    embedding_layer=tl.layers.Embedding(vocabulary_size=vocabulary_size, embedding_size=emb_dim),
)

NameError: name 'decoder_seq_length' is not defined